# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
cwd = Path.cwd()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# Data Challenge

The City of Goodeats is looking for ways to improve **1) how it conducts food safety inspections for food establishments (e.g. restaurants); and 2) how food establishments perform on the inspections.** The Commissioner has come to you for help. She has provided you with two datasets: “food establishment violations” and “food establishment grades.” “Food establishment violations” records every violation for every inspection on every food establishment in the City (e.g. level=restaurant—inspection—violation). "Food establishment grades” scores each inspection based on the number and severity of the violations observed and presents a score and grade for each inspection performed (level=restaurant—inspection). The Commissioner would like you to use the data to generate insights into the City’s inspection process and/or inspection results that she can use to make improvements. Please choose one (1) of the bullets below and answer the questions using one or both datasets:
 
* Is the City conducting inspections effectively? Are poorer performing establishments inspected more often than better performing establishments? What are the trends over time? 
* Are there commonalities or trends among establishments that perform poorly?
* How do establishments perform on re-inspections?  How long does it take after an initial inspection for establishments to correct all violations?
*	Do establishments improve their performance on inspections over time? 
*	What changes, if any, would you make to the scoring/grading system to better reflect inspection performance?
 
**Prepare a brief memo or presentation** for the Commissioner. Include a **short explanation of how you approached the question(s), a statement of your main finding(s), and a set of data visualizations that show your analysis.** You may visualize your analysis in any form or medium you choose and use any tools or applications you wish, but please complete any coding work in either R or Python.


## Grades Exploratory analysis

In [2]:
grades = pd.read_csv(cwd / 'food+establishment+grades.csv', parse_dates=['RESULTDTTM'], infer_datetime_format=True)
grades = grades.fillna('-').replace(' ','-')
grades['ISSDTTM'] = pd.to_datetime(grades['ISSDTTM'], errors='coerce')
grades['EXPDTTM'] = pd.to_datetime(grades['EXPDTTM'], errors='coerce')

In [3]:
grades.groupby(['LICENSENO', 'GRADE', 'RESULTDTTM', 'RESULT', 'SCORE'])['SCORE'].count().head(50)
grades.groupby(['LICENSENO', 'RESULTDTTM', 'RESULT'])['RESULT'].count()
grades['year'] = pd.DatetimeIndex(grades['RESULTDTTM']).year
grades.groupby(['LICENSENO', 'year', 'RESULT'])['RESULT'].count().head(100)

LICENSENO  year  RESULT    
54         2012  HE_NotReq     1
4032       2012  HE_NotReq     1
4036       2012  HE_Filed      3
           2013  HE_Fail       1
                 HE_FailExt    1
                 HE_Pass       1
           2014  HE_Filed      1
           2015  HE_Fail       1
                 HE_FailExt    1
                 HE_Filed      1
           2016  HE_Fail       1
                 HE_Filed      1
                 HE_Pass       1
           2017  HE_Filed      2
           2018  HE_Fail       1
                 HE_Filed      1
                 HE_Hearing    1
17602      2012  HE_Fail       2
                 HE_NotReq     1
                 HE_Pass       1
           2013  HE_Pass       1
           2016  HE_Pass       1
           2017  HE_Filed      1
           2018  HE_Pass       1
17603      2012  HE_NotReq     1
17605      2012  HE_NotReq     1
17606      2012  HE_NotReq     1
17607      2012  HE_NotReq     1
17608      2012  HE_NotReq     1
17609      2012

**Which features contain blank, null or empty values?**

Using `.info()` there are no null values originally. Converting ISSDTTM, EXPDTTM to datetime variables created NaT values where there were blank/empty ones before. No other columns have blank values.

**What are the data types for various features?**

* Three features are integer or floats.
* Five features are strings (object).
* Three features are datetime.

**What is the distribution of categorical features?**

*LICSTATUS* variable has three possible values (Active, Inactive and Deleted) with 
* 75.6% active
* 24.4% inactive 
* Only 11/72243 are Deleted.

*LICENSECAT* has four possible values ('FT', 'FS', 'RF', 'MFW') with 
* 37.3% FT
* 42.6% FS
* 18% RF
* 1.9% MFW.

*RESULT* has eight possible values ('HE_Pass', 'HE_Filed', 'HE_Fail', 'HE_FailExt', 'HE_Hearing', 'HE_OutBus', 'HE_TSOP', 'HE_NotReq')
*GRADE* has three possible values ('A', 'B', 'C'), 
* A has 69%
* B 14.5%

**Mean, Median, Mode of qualitative data**

*SUM_VIOLATIONS* 
* mean 8.642346
* median 0.000000

*SCORE* 
* mean 91.357654
* median 100.000000
* mode 100

### Poor results for establishments

In [4]:
bad = ['HE_Fail', 'HE_FailExt', 'HE_Hearing', 'HE_OutBus', 'HE_TSOP', 'HE_Closure', 'HE_Misc', 'HE_FAILNOR']
ending = ['HE_OutBus', 'HE_TSOP']
bad_grades = grades[grades['RESULT'].isin(bad)]
bad_grades.sort_values('LICENSENO').head(20)
bad_grades_A = bad_grades[bad_grades['GRADE'] == 'A']
bad_grades_B = bad_grades[bad_grades['GRADE'] == 'B']
bad_grades_C = bad_grades[bad_grades['GRADE'] == 'C']


bad_grades_FT = bad_grades[bad_grades['LICENSECAT'] == 'FT']
bad_grades_FS = bad_grades[bad_grades['LICENSECAT'] == 'FS']
bad_grades_MFW = bad_grades[bad_grades['LICENSECAT'] == 'MFW']
bad_grades_RF = bad_grades[bad_grades['LICENSECAT'] == 'RF']

bad_grades_active = bad_grades[bad_grades['LICSTATUS'] == 'Active']
bad_grades_inactive = bad_grades[bad_grades['LICSTATUS'] == 'Inactive']
bad_grades_deleted = bad_grades[bad_grades['LICSTATUS'] == 'Deleted']
len(bad_grades_C)/len(bad_grades)

grou = bad_grades.groupby(['SCORE'])['GRADE'].count().sort_values()
grou.tail()
bad_grades['SCORE'].value_counts(normalize=True)

 90     0.069257
 88     0.061937
 98     0.059047
 94     0.047560
 86     0.047523
 96     0.046884
 92     0.043431
 84     0.039039
 80     0.033033
 82     0.030293
 78     0.029317
 76     0.024399
 74     0.021734
 100    0.017530
 79     0.016517
 81     0.016441
 72     0.015691
 70     0.014977
 89     0.014827
 66     0.014489
 68     0.014452
 83     0.014452
 87     0.014377
 77     0.014377
 85     0.013551
 91     0.012200
 75     0.011974
 73     0.011562
 64     0.011186
 71     0.010360
 69     0.010060
 67     0.009647
 58     0.009047
 60     0.009047
 65     0.008709
 62     0.008446
 63     0.008108
 93     0.007920
 56     0.007245
 61     0.006869
 59     0.006832
 54     0.006381
 57     0.005818
 52     0.004992
 55     0.004992
 53     0.004842
 50     0.004767
 51     0.004467
 49     0.004429
 48     0.004129
 46     0.003829
 47     0.003378
 42     0.003228
 44     0.003003
 45     0.002890
 43     0.002853
 41     0.002703
 38     0.002628
 40     0.0025

In [5]:
grades[(grades['RESULT'] == 'HE_Pass') & (grades['GRADE'] == 'C')]
bad_grades.head()

,LICENSENO,ISSDTTM,EXPDTTM,LICSTATUS,LICENSECAT,DESCRIPT,RESULT,RESULTDTTM,SUM_VIOLATIONS,SCORE,GRADE,year
2,18086,2011-12-27 12:46:00,2011-12-31 23:59:00,Inactive,RF,Retail Food,HE_Fail,2012-01-03 10:06:00,6,94,A,2012
3,77035,2012-01-26 11:01:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_FailExt,2012-01-03 10:07:00,12,88,B,2012
6,76109,2012-07-19 12:58:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_Fail,2012-01-03 10:44:00,16,84,B,2012
8,25877,2012-01-03 14:09:00,2018-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Fail,2012-01-03 10:47:00,32,68,C,2012
11,21466,2012-02-06 13:35:00,2018-12-31 23:59:00,Active,FT,Eating & Drinking w/ Take Out,HE_Fail,2012-01-03 10:53:00,8,92,B,2012


In [6]:
grades_by_cat = grades.groupby(['RESULT'])['RESULT'].count().sort_values()
grades_by_cat.plot(kind='bar')

In [7]:
grades[(grades['SCORE'] < 0) & (grades['LICSTATUS'] == 'Inactive')]

,LICENSENO,ISSDTTM,EXPDTTM,LICSTATUS,LICENSECAT,DESCRIPT,RESULT,RESULTDTTM,SUM_VIOLATIONS,SCORE,GRADE,year
875,34762,2012-01-05 12:45:00,2012-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_TSOP,2012-02-01 13:41:00,115,-15,C,2012
924,21828,2012-01-25 15:00:00,2015-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Fail,2012-02-03 10:35:00,114,-14,C,2012
1368,30345,2012-06-15 10:40:00,2012-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_TSOP,2012-02-17 10:04:00,104,-4,C,2012
1528,37220,2011-12-12 09:07:00,2015-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_TSOP,2012-02-24 12:15:00,105,-5,C,2012
2183,28997,2012-01-31 17:29:00,2014-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2012-03-19 11:22:00,106,-6,C,2012
2559,30704,2012-02-15 10:21:00,2015-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_TSOP,2012-03-29 14:00:00,165,-65,C,2012
2834,26464,2012-02-15 10:32:00,2015-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2012-04-06 16:22:00,114,-14,C,2012
3195,77481,2011-11-25 14:13:00,2014-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Fail,2012-04-19 13:41:00,196,-96,C,2012
3800,22753,2012-02-17 10:30:00,2014-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Fail,2012-05-08 10:51:00,130,-30,C,2012
4237,32186,2011-12-30 15:12:00,2016-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Fail,2012-05-18 13:29:00,114,-14,C,2012


### Splitting up grades by year

In [8]:
grades['RESULTDTTM'].min()
grades_2012 = grades[grades['RESULTDTTM'].dt.year == 2012]
grades_2013 = grades[grades['RESULTDTTM'].dt.year == 2013]
grades_2014 = grades[grades['RESULTDTTM'].dt.year == 2014]
grades_2015 = grades[grades['RESULTDTTM'].dt.year == 2015]
grades_2016 = grades[grades['RESULTDTTM'].dt.year == 2016]
grades_2017 = grades[grades['RESULTDTTM'].dt.year == 2017]
grades_2018 = grades[grades['RESULTDTTM'].dt.year == 2018]

## Violations Exploratory Analysis

In [9]:
violations = pd.read_csv(cwd / 'food+establishment+violations.csv', parse_dates=['RESULTDTTM'])
violations = violations.fillna('-').replace(' ','-')
violations['ISSDTTM'] = pd.to_datetime(violations['ISSDTTM'], errors='coerce')
violations['EXPDTTM'] = pd.to_datetime(violations['EXPDTTM'], errors='coerce')

In [10]:
violations.head()

,LICENSENO,ISSDTTM,EXPDTTM,LICSTATUS,LICENSECAT,DESCRIPT,RESULT,RESULTDTTM,VIOLATION,VIOLLEVEL,VIOLDESC,VIOLDTTM,VIOLSTATUS,PROPERTY_ID,LOCATION
0,77958,2012-03-14 09:17:00,2012-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2012-01-03 07:26:00,-,-,-,-,-,155968,"(42.355863000, -71.061898000)"
1,74470,2012-02-01 12:29:00,2018-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Filed,2012-01-03 09:42:00,15-4-202.16,*,Non-Food Contact Surfaces,1/3/2012 9:42,Fail,48511,"(42.278028000, -71.066046000)"
2,74470,2012-02-01 12:29:00,2018-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Filed,2012-01-03 09:42:00,36-6-501.11-.12,*,Improper Maintenance of Floors,1/3/2012 9:42,Fail,48511,"(42.278028000, -71.066046000)"
3,74470,2012-02-01 12:29:00,2018-12-31 23:59:00,Active,FS,Eating & Drinking,HE_Filed,2012-01-03 09:42:00,37-6-501.11-.12,*,Improper Maintenance of Walls/Ceilings,1/3/2012 9:42,Fail,48511,"(42.278028000, -71.066046000)"
4,18086,2011-12-27 12:46:00,2011-12-31 23:59:00,Inactive,RF,Retail Food,HE_Fail,2012-01-03 10:06:00,23-4-602.13,*,Non-Food Contact Surfaces Clean,1/3/2012 10:06,Fail,14788,"(42.382370000, -71.018390000)"


**Which features contain blank, null or empty values?**

Some functions in python, specifically pandas, will disregard entire rows from groupbys and joins if even one value is NAN or missing.

* VIOLATIONS, VIOLLEVEL, VIOLDESC, VIOLDTTM, VIOLSTATUS, PROPERTY_ID, LOCATION have null or missing values.

**What are the data types for various features?**

* Two features are integer or floats.
* Ten features are strings (object).
* Three features are datetime

### 1/3 of all violations were

In [11]:
violations.groupby(['LICENSECAT', 'RESULT'])['LICENSENO'].count().sort_values(ascending=False).head(20)

LICENSECAT  RESULT    
FS          HE_Fail       67870
FT          HE_Fail       52264
FS          HE_Pass       46043
FT          HE_Pass       37981
FS          HE_Filed      18063
RF          HE_Fail       15197
FT          HE_Filed      13658
FS          HE_FailExt    13469
RF          HE_Pass       13158
FT          HE_FailExt    9804 
FS          HE_Hearing    6419 
RF          HE_Filed      6030 
FT          HE_Hearing    5000 
FS          HE_NotReq     3109 
RF          HE_FailExt    2925 
FT          HE_NotReq     2606 
FS          HE_TSOP       1690 
RF          HE_NotReq     1645 
MFW         HE_Pass       1515 
FT          HE_TSOP       1441 
Name: LICENSENO, dtype: int64

In [12]:
violations_inspection = violations.groupby(['LICENSENO', 'RESULTDTTM', 'VIOLSTATUS'])['VIOLSTATUS'].count()
violations_inspection.head(100)

LICENSENO  RESULTDTTM           VIOLSTATUS
54         2012-12-30 00:00:00  -             1 
4032       2012-12-30 00:00:00  -             1 
4036       2012-01-13 13:15:00  Fail          2 
           2012-07-20 12:47:00  Fail          2 
           2012-12-03 13:08:00  Fail          3 
           2013-05-30 10:34:00  Fail          2 
           2013-06-18 10:52:00  -             1 
                                Fail          2 
           2013-06-21 11:28:00  Pass          2 
           2014-05-19 10:33:00  Fail          2 
           2015-07-28 14:17:00  Fail          7 
           2015-08-04 14:12:00  Fail          6 
                                Pass          1 
           2015-09-04 12:04:00  Fail          1 
                                Pass          5 
           2016-02-25 13:42:00  Fail          5 
           2016-03-14 12:08:00  Pass          5 
           2016-09-30 13:24:00  Fail          2 
           2017-03-06 14:23:00  Fail          3 
           2017-09-20 12:0

### Most common violations

In [13]:
violations.groupby(['VIOLDESC', 'RESULT'])['RESULT'].count().sort_values(ascending=False).head(20)

VIOLDESC                                RESULT   
-                                       HE_Pass      11237
Non-Food Contact Surfaces Clean         HE_Fail      10724
Improper Maintenance of Walls/Ceilings  HE_Fail      10281
Non-Food Contact Surfaces               HE_Fail      8765 
Improper Maintenance of Floors          HE_Fail      8225 
Food Protection                         HE_Fail      7765 
-                                       HE_NotReq    7252 
Non-Food Contact Surfaces Clean         HE_Pass      6992 
Food Contact Surfaces Clean             HE_Fail      6959 
Improper Maintenance of Walls/Ceilings  HE_Pass      6688 
Hand Cleaner  Drying  Tissue Signage    HE_Fail      5611 
Non-Food Contact Surfaces               HE_Pass      5546 
Premises Maintained                     HE_Fail      5209 
Food Protection                         HE_Pass      5068 
Improper Maintenance of Floors          HE_Pass      4963 
Installed and Maintained                HE_Fail      4570 
Food C

## **Are there commonalities or trends among establishments that perform poorly?**

**What does 'perform poorly' mean?**
* any establishment that gets a result of 'HE_Fail', 'HE_FailExt', 'HE_Hearing', 'HE_OutBus', 'HE_TSOP' at least once. I did not include HE_Filed in this estimate because although

**type of establishment?**
* FT 38.6% bad_grades_FT / bad_grades
* FS 46%
* MFW 1.7%
* RF 13.7%

**how many are repeat offenders?**
below

**when were they licensed/evaluated?**

**how many violations**

**severity of violations**

**active/inactive/deleted?**

*LICSTATUS*
* active 78.6% bad_grades_active / bad_grades
* inactive 23.1%
* deleted 0.02%

**mean, median, mode compare individual**

*SUM_VIOLATIONS*
* mean 22.285323 bad_grades
* median 17
* mode 10, 12,2 , 6 , 14

*SCORE*
* mean 77.714677
* median 83

**most/least common violation**

**percentage of offenders that got _ grade**
* A 17.1% bad_grades_A / bad_grades
* B 38.5%
* C 44.4%

**in a real time, i would google around more about each violation code to see if there are commonalities between each code**

In [14]:
violations_inspection = violations.groupby(['LICENSENO', 'RESULTDTTM', 'VIOLSTATUS', 'VIOLLEVEL'])['VIOLSTATUS'].count()
violations_inspection.head(20)

LICENSENO  RESULTDTTM           VIOLSTATUS  VIOLLEVEL
54         2012-12-30 00:00:00  -           -            1
4032       2012-12-30 00:00:00  -           -            1
4036       2012-01-13 13:15:00  Fail        *            2
           2012-07-20 12:47:00  Fail        *            2
           2012-12-03 13:08:00  Fail        *            3
           2013-05-30 10:34:00  Fail        *            2
           2013-06-18 10:52:00  -           -            1
                                Fail        *            2
           2013-06-21 11:28:00  Pass        *            2
           2014-05-19 10:33:00  Fail        *            2
           2015-07-28 14:17:00  Fail        *            7
           2015-08-04 14:12:00  Fail        *            6
                                Pass        *            1
           2015-09-04 12:04:00  Fail        *            1
                                Pass        *            5
           2016-02-25 13:42:00  Fail        *            5
  

In [15]:
violations_severity = violations.groupby(['LICENSENO', 'VIOLLEVEL'])['VIOLLEVEL'].count().sort_values(ascending=False)
violations_severity.head(20)

LICENSENO  VIOLLEVEL
24748      *            527
74662      *            379
31241      *            364
22388      *            344
34592      *            341
18686      *            333
26357      *            329
75844      *            317
33361      *            316
21037      *            313
30908      *            305
31259      *            293
29259      *            293
37129      *            288
24709      *            287
74834      *            285
23387      *            278
22464      *            275
23829      *            275
137896     *            275
Name: VIOLLEVEL, dtype: int64

In [16]:
good = ['HE_Pass', 'Pass']
bad_violations = violations[(violations['RESULT'].isin(bad))]
bad_violations.groupby(['PROPERTY_ID', 'LICENSENO', 'RESULTDTTM', 'RESULT', 'VIOLSTATUS'])['RESULT'].count().tail(20)

PROPERTY_ID  LICENSENO  RESULTDTTM           RESULT      VIOLSTATUS
-            160578     2017-11-27 13:28:00  HE_TSOP     Fail          7 
             171026     2017-04-24 12:48:00  HE_Fail     Fail          3 
             171399     2017-05-01 13:11:00  HE_Fail     Fail          2 
             172338     2016-05-13 10:35:00  HE_Fail     Fail          3 
                        2016-10-27 13:58:00  HE_Fail     Fail          7 
                        2017-05-15 13:39:00  HE_Fail     Fail          8 
                        2018-06-12 10:43:00  HE_Fail     Fail          4 
             308511     2017-04-27 09:00:00  HE_Fail     Fail          10
                        2017-08-22 11:27:00  HE_Fail     Fail          8 
             332280     2017-11-02 13:18:00  HE_Fail     Fail          1 
                        2018-02-16 11:20:00  HE_Closure  Fail          5 
             345290     2017-08-24 08:07:00  HE_Fail     Fail          3 
             345291     2017-08-24 08:15:00 

In [17]:
bad_violations.groupby(['RESULT'])['RESULT'].count().sort_values()

RESULT
HE_Misc       27    
HE_FAILNOR    39    
HE_Closure    346   
HE_OutBus     646   
HE_TSOP       3570  
HE_Hearing    12788 
HE_FailExt    26452 
HE_Fail       136721
Name: RESULT, dtype: int64

### severity of violations - 1/3 of all violations were non-critical

In [18]:
violations_codes = violations.groupby(['VIOLATION', 'VIOLLEVEL', 'VIOLSTATUS'])['RESULT'].count().sort_values(ascending=False)
bad_violations_levels = bad_violations.groupby(['VIOLLEVEL', 'RESULT'])['RESULT'].count().sort_values(ascending=False)
bad_violations_levels

VIOLLEVEL  RESULT    
*          HE_Fail       102342
***        HE_Fail       25471 
*          HE_FailExt    19690 
           HE_Hearing    9066  
**         HE_Fail       8843  
***        HE_FailExt    5116  
           HE_Hearing    2913  
*          HE_TSOP       2200  
**         HE_FailExt    1626  
***        HE_TSOP       1050  
**         HE_Hearing    800   
*          HE_OutBus     281   
**         HE_TSOP       278   
-          HE_OutBus     261   
*          HE_Closure    196   
***        HE_Closure    125   
           HE_OutBus     82    
-          HE_Fail       65    
           HE_TSOP       42    
*          HE_FAILNOR    32    
-          HE_Misc       23    
**         HE_OutBus     22    
-          HE_FailExt    20    
**         HE_Closure    17    
-          HE_Hearing    9     
           HE_Closure    7     
***        HE_FAILNOR    5     
*          HE_Misc       3     
**         HE_FAILNOR    2     
           HE_Misc       1     
1919       HE_Clos

In [19]:
violations[violations['VIOLATION'] == '-'].head(20)

,LICENSENO,ISSDTTM,EXPDTTM,LICSTATUS,LICENSECAT,DESCRIPT,RESULT,RESULTDTTM,VIOLATION,VIOLLEVEL,VIOLDESC,VIOLDTTM,VIOLSTATUS,PROPERTY_ID,LOCATION
0,77958,2012-03-14 09:17:00,2012-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2012-01-03 07:26:00,-,-,-,-,-,155968,"(42.355863000, -71.061898000)"
125,21703,2011-12-08 10:48:00,2015-12-31 23:59:00,Inactive,RF,Retail Food,HE_Pass,2012-01-03 11:44:00,-,-,-,-,-,-,-
132,75511,2012-02-14 13:16:00,2018-12-31 23:59:00,Active,FT,Eating & Drinking w/ Take Out,HE_Pass,2012-01-03 11:57:00,-,-,-,-,-,41097,"(42.348492000, -71.096725000)"
161,20344,2012-01-18 16:35:00,2014-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Filed,2012-01-03 12:32:00,-,-,-,-,-,156595,"(42.361959000, -71.142208000)"
216,22035,2012-01-12 14:31:00,2018-12-31 23:59:00,Inactive,RF,Retail Food,HE_Pass,2012-01-03 14:11:00,-,-,-,-,-,157731,"(42.253484000, -71.124850000)"
229,22311,2011-12-30 13:53:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_Pass,2012-01-03 14:24:00,-,-,-,-,-,-,-
236,77980,2012-03-14 09:15:00,2013-12-31 23:59:00,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,2012-01-04 09:15:00,-,-,-,-,-,63353,"(42.299330000, -71.064790000)"
268,19424,2012-02-15 13:45:00,2017-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Pass,2012-01-04 10:46:00,-,-,-,-,-,156514,"(42.318168000, -71.056920000)"
276,77953,2012-01-10 09:32:00,2012-12-31 23:59:00,Inactive,FS,Eating & Drinking,HE_Pass,2012-01-04 10:59:00,-,-,-,-,-,156924,"(42.345142000, -71.086221000)"
286,67909,2012-02-23 11:27:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_FailExt,2012-01-04 11:14:00,-,-,-,-,-,157477,"(42.270761000, -71.074469000)"


In [20]:
violations[violations['PROPERTY_ID'] == 0].head(20)

,LICENSENO,ISSDTTM,EXPDTTM,LICSTATUS,LICENSECAT,DESCRIPT,RESULT,RESULTDTTM,VIOLATION,VIOLLEVEL,VIOLDESC,VIOLDTTM,VIOLSTATUS,PROPERTY_ID,LOCATION
946,67795,2012-01-17 16:37:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_Fail,2012-01-09 14:45:00,29-5-201/02.11,*,Installed and Maintained,1/9/2012 14:45,Fail,0,-
947,67795,2012-01-17 16:37:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_Fail,2012-01-09 14:45:00,33-5-501.13-.17,*,Adequate Number Frequency Vermin Proof,1/9/2012 14:45,Fail,0,-
948,67795,2012-01-17 16:37:00,2018-12-31 23:59:00,Active,RF,Retail Food,HE_Fail,2012-01-09 14:45:00,42-6-501.113/.114,*,Premises Maintained,1/9/2012 14:45,Fail,0,-
984,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,08-3-305-307.11,*,Food Protection,1/10/2012 10:27,Pass,0,-
985,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,14-4-202.11,*,Food Contact Surfaces Design,1/10/2012 10:27,Pass,0,-
986,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,22-4-601/602.11,**,Food Contact Surfaces Clean,1/10/2012 10:27,Pass,0,-
987,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,23-4-602.13,*,Non-Food Contact Surfaces Clean,1/10/2012 10:27,Pass,0,-
988,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,27-5-103.11-.12,***,Hot and Cold Water,1/10/2012 10:27,Pass,0,-
989,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,41-7-102.11,***,Labeled Common Name,1/10/2012 10:27,Pass,0,-
990,75234,2011-12-23 12:40:00,2017-12-31 23:59:00,Inactive,MFW,Mobile Food Walk On,HE_Pass,2012-01-10 10:27:00,42-6-501.113/.114,*,Premises Maintained,1/10/2012 10:27,Pass,0,-


### violations by location

In [21]:
violations.groupby(['LOCATION', 'PROPERTY_ID'])['RESULT'].count().sort_values(ascending=False).head(20)

LOCATION                       PROPERTY_ID
-                              -              64527
                               0.0            5592 
(42.352411000, -71.125329000)  157732.0       1057 
(42.355830000, -71.060400000)  157146.0       868  
(42.347296000, -71.081457000)  157244.0       845  
(42.330446000, -71.082030000)  157231.0       697  
(42.346723000, -71.098680000)  156417.0       686  
(42.360036000, -71.056253000)  341035.0       660  
(42.329370000, -71.084460000)  144493.0       655  
(42.353081000, -71.130624000)  20067.0        637  
(42.346820000, -71.075430000)  46311.0        551  
(42.287460000, -71.150160000)  27711.0        547  
-                              341353.0       522  
(42.265164000, -71.099382000)  116675.0       519  
(42.351362000, -71.068740000)  158173.0       503  
(42.358200000, -71.062380000)  10099.0        500  
(42.349380000, -71.084014000)  101491.0       490  
(42.351362000, -71.068740000)  155883.0       468  
(42.292072000, -71.07

### most common type of violations

In [22]:
violations['year'] = pd.DatetimeIndex(violations['RESULTDTTM']).year
violations.groupby(['VIOLDESC', 'VIOLLEVEL', 'LICENSECAT', 'RESULT'])['RESULT'].count().sort_values(ascending=False).head(20)

VIOLDESC                                VIOLLEVEL  LICENSECAT  RESULT   
Improper Maintenance of Walls/Ceilings  *          FS          HE_Fail      5459
Non-Food Contact Surfaces Clean         *          FS          HE_Fail      4910
Non-Food Contact Surfaces               *          FS          HE_Fail      4491
Non-Food Contact Surfaces Clean         *          FT          HE_Fail      4275
Improper Maintenance of Floors          *          FS          HE_Fail      3844
-                                       -          FT          HE_Pass      3837
Improper Maintenance of Walls/Ceilings  *          FT          HE_Fail      3798
Food Protection                         *          FS          HE_Fail      3747
-                                       -          FS          HE_Pass      3560
Improper Maintenance of Walls/Ceilings  *          FS          HE_Pass      3487
-                                       -          RF          HE_Pass      3478
Non-Food Contact Surfaces           

In [23]:
violations['RESULT'].unique()

array(['HE_Pass', 'HE_Filed', 'HE_Fail', 'HE_FailExt', 'HE_Hearing',
       'HE_OutBus', 'HE_TSOP', 'HE_Closure', 'HE_NotReq', 'HE_Misc',
       'HE_FAILNOR', 'Pass'], dtype=object)